Function random iri parts

In [1]:
import numpy as np
import pandas as pd

def generate_parts(target_value, num_parts, tolerance=0.01):
    total_sum = target_value * num_parts
    while True:
        # Generate random parts
        parts = np.random.uniform(low=total_sum / num_parts * 0.9, high=total_sum / num_parts * 1.1, size=num_parts)
        # Ensure the sum is correct
        if np.abs(np.sum(parts) - total_sum) < tolerance:
            return parts

target_value = 4.182634
num_parts = 4

# Generate parts
parts = generate_parts(target_value, num_parts)

# Create DataFrame
data = {
    'old_value': [target_value] * num_parts,
    'iri_random': parts
}
df = pd.DataFrame(data)

Convert iri 20 to 5 and random iri in 5m and join iri and rut all .csv file 

In [85]:
import os
import fnmatch
import numpy as np
import pandas as pd

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Define the path to your directories
path = r"D:\xenomatixs\output\survey_data_20240726\Output"
pic = r"D:\xenomatixs\output\survey_data_20240726\PAVE"

def split_and_randomize(value, parts=4):
    random_parts = np.random.rand(parts)
    random_parts /= random_parts.sum()  # Normalize to ensure sum is equal to 1
    split_values = random_parts * value  # Scale fractions to match the original value
    return split_values

def generate_parts(target_values, num_parts, tolerance):
    parts_list = []
    for target_value in target_values:
        total_sum = target_value * num_parts
        while True:
            # Generate random parts
            parts = np.random.uniform(low=total_sum / num_parts * 0.9, high=total_sum / num_parts * 1.1, size=num_parts)
            # Ensure the sum is correct
            if np.abs(np.sum(parts) - total_sum) < tolerance:
                parts_list.append(parts)
                break
    return parts_list

# Find all relevant CSV files and process them
def process_csv_files(path):
    iri_dataframes = {}
    rutting_dataframes = {}

    for root, dirs, files in os.walk(path):
        # Find files
        iri_files = [f for f in files if f.endswith('.csv') and 'xw_iri_qgis' in f]
        rutting_files = [f for f in files if f.endswith('.csv') and 'xw_rutting' in f]
        
        # Process 'xw_iri_qgis' files
        for filename in iri_files:
            file_path = os.path.join(root, filename)
            iri_df = pd.read_csv(file_path, delimiter=';')
            iri_df.columns = iri_df.columns.str.strip()
            survey_code = filename.split('_')[3].split('.')[0]
            iri_df['survey_code'] = survey_code
            iri_df['iri'] = (iri_df['iri left (m/km)'] + iri_df['iri right (m/km)']) / 2        
            iri_df.drop(columns=['geometry'], errors='ignore', inplace=True)
            
            # Generate random values
            target_values = iri_df['iri']
            num_parts = 4
            tolerance = 0.3
            parts_list = generate_parts(target_values, num_parts, tolerance)

            # Expand DataFrame by repeating the rows
            iri_df = iri_df.loc[iri_df.index.repeat(num_parts)].reset_index(drop=True)
            iri_df['iri_lane'] = np.concatenate(parts_list)
            
            # Set initial event columns
            increment = 5 if fnmatch.fnmatch(filename, '*xw_iri_qgis*') else 5
            iri_df['chainage'] = iri_df.index * 5
            iri_df['event_start'] = range(0, len(iri_df) * increment, increment)
            iri_df['event_end'] = iri_df['event_start'] + increment
            iri_dataframes[filename] = iri_df
            
            print(f"Updated {filename} into IRI DataFrame.")
        
        # Process 'xw_rutting' files
        for filename in rutting_files:
            file_path = os.path.join(root, filename)
            rut_df = pd.read_csv(file_path, delimiter=';')
            rut_df.columns = rut_df.columns.str.strip()
            rut_df.drop(columns=['Unnamed: 5'], inplace=True, errors='ignore')
            increment = 5 if fnmatch.fnmatch(filename, '*xw_rutting*') else 5
            rut_df['event_start'] = range(0, len(rut_df) * increment, increment)
            rut_df['event_end'] = rut_df['event_start'] + increment
            
            survey_code = filename.split('_')[2].split('.')[0]
            rut_df['index'] = rut_df.index * 25 // 5
            rut_df.set_index('index', inplace=True)
            rut_df['survey_code'] = survey_code
            rut_df['rut_point_x'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[1]))
            rut_df['rut_point_y'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[0]))
            rut_df['rut_point_x'].fillna(0, inplace=True)
            rut_df['rut_point_y'].fillna(0, inplace=True)
        
            rut_df.rename(columns={'left rutting height': 'left_rutting', 'right rutting height': 'right_rutting', 'average height': 'avg_rutting'}, inplace=True)
            rut_df.drop(columns=['qgis_shape'], inplace=True)
            rutting_dataframes[filename] = rut_df

            print(f"Updated {filename} into Rutting DataFrame.")

    return iri_dataframes, rutting_dataframes

# Perform the left join on xw_rutting and xw_iri_qgis
def left_join_dataframes(df_rutting, df_iri):
    joined_df = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end', 'survey_code'], suffixes=('_rutting', '_iri'))
    return joined_df

# Perform jpg file and frame number
def get_jpg_names_and_nums(directory):
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)

    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace('_', 'RUN0')

    return jpg_files, frame_df

# use

jpg_files, frame_df = get_jpg_names_and_nums(pic)
iri_dataframes, rutting_dataframes = process_csv_files(path)

joined_dataframes = {}
for rutting_file in rutting_dataframes:
    for iri_file in iri_dataframes:
        if 'xw_rutting' in rutting_file and 'xw_iri_qgis' in iri_file:
            joined_df = left_join_dataframes(rutting_dataframes[rutting_file], iri_dataframes[iri_file])

            # Group by survey_code and calculate min and max chainage
            grouped_df = joined_df.groupby('survey_code').agg(
                max_chainage=('chainage', 'max'),
                min_chainage=('chainage', 'min')
            ).reset_index()

            joined_df = pd.merge(joined_df, grouped_df, on='survey_code', how='left')
            joined_df = pd.merge(joined_df, frame_df, on='survey_code', how='left')
            joined_df['frame_num'] = joined_df.groupby(['survey_code', 'pic_count']).cumcount() + 1
            joined_df['chainage_pic'] = round(joined_df['max_chainage'] / joined_df['pic_count'] * joined_df['frame_num'],0)
            joined_dataframes[f"{rutting_file}_{iri_file}"] = joined_df
            
final_df = pd.concat(joined_dataframes.values(), ignore_index=True)
final_df = final_df[final_df['iri'].notnull()]
final_df.to_csv('final_df.csv', index=False)

Updated xw_iri_qgis_20240726RUN01.csv into IRI DataFrame.
Updated xw_rutting_20240726RUN01.csv into Rutting DataFrame.
Updated xw_iri_qgis_20240726RUN02.csv into IRI DataFrame.
Updated xw_rutting_20240726RUN02.csv into Rutting DataFrame.
Updated xw_iri_qgis_20240726RUN03.csv into IRI DataFrame.
Updated xw_rutting_20240726RUN03.csv into Rutting DataFrame.


Combine above and below 

In [1]:
import os
import fnmatch
import numpy as np
import pandas as pd

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Define the path to your directories
path = r"D:\xenomatixs\output\survey_data_20240726\Output"
pic = r"D:\xenomatixs\output\survey_data_20240726\PAVE"
log = r'D:\xenomatixs\output\survey_data_20240726\Output'

def split_and_randomize(value, parts=4):
    random_parts = np.random.rand(parts)
    random_parts /= random_parts.sum()  # Normalize to ensure sum is equal to 1
    split_values = random_parts * value  # Scale fractions to match the original value
    return split_values

def generate_parts(target_values, num_parts, tolerance):
    parts_list = []
    for target_value in target_values:
        total_sum = target_value * num_parts
        while True:
            # Generate random parts
            parts = np.random.uniform(low=total_sum / num_parts * 0.9, high=total_sum / num_parts * 1.1, size=num_parts)
            # Ensure the sum is correct
            if np.abs(np.sum(parts) - total_sum) < tolerance:
                parts_list.append(parts)
                break
    return parts_list

# Find all relevant CSV files and process them
def process_csv_files(path):
    iri_dataframes = {}
    rutting_dataframes = {}

    for root, dirs, files in os.walk(path):
        # Find files
        iri_files = [f for f in files if f.endswith('.csv') and 'xw_iri_qgis' in f]
        rutting_files = [f for f in files if f.endswith('.csv') and 'xw_rutting' in f]
        
        # Process 'xw_iri_qgis' files
        for filename in iri_files:
            file_path = os.path.join(root, filename)
            iri_df = pd.read_csv(file_path, delimiter=';')
            iri_df.columns = iri_df.columns.str.strip()
            survey_code = filename.split('_')[3].split('.')[0]
            iri_df['survey_code'] = survey_code
            iri_df['iri'] = (iri_df['iri left (m/km)'] + iri_df['iri right (m/km)']) / 2        
            iri_df.drop(columns=['geometry'], errors='ignore', inplace=True)
            
            # Generate random values
            target_values = iri_df['iri']
            num_parts = 4
            tolerance = 0.3
            parts_list = generate_parts(target_values, num_parts, tolerance)

            # Expand DataFrame by repeating the rows
            iri_df = iri_df.loc[iri_df.index.repeat(num_parts)].reset_index(drop=True)
            iri_df['iri_lane'] = np.concatenate(parts_list)
            
            # Set initial event columns
            increment = 5 if fnmatch.fnmatch(filename, '*xw_iri_qgis*') else 5
            iri_df['chainage'] = iri_df.index * 5
            iri_df['event_start'] = range(0, len(iri_df) * increment, increment)
            iri_df['event_end'] = iri_df['event_start'] + increment
            iri_dataframes[filename] = iri_df
            
            print(f"Updated {filename} into IRI DataFrame.")
        
        # Process 'xw_rutting' files
        for filename in rutting_files:
            file_path = os.path.join(root, filename)
            rut_df = pd.read_csv(file_path, delimiter=';')
            rut_df.columns = rut_df.columns.str.strip()
            rut_df.drop(columns=['Unnamed: 5'], inplace=True, errors='ignore')
            increment = 5 if fnmatch.fnmatch(filename, '*xw_rutting*') else 5
            rut_df['event_start'] = range(0, len(rut_df) * increment, increment)
            rut_df['event_end'] = rut_df['event_start'] + increment
            
            survey_code = filename.split('_')[2].split('.')[0]
            rut_df['index'] = rut_df.index * 25 // 5
            rut_df.set_index('index', inplace=True)
            rut_df['survey_code'] = survey_code
            rut_df['rut_point_x'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[1]))
            rut_df['rut_point_y'] = rut_df['qgis_shape'].apply(lambda x: float(x.split('(')[1].split(')')[0].split(',')[0].split(' ')[0]))
            rut_df['rut_point_x'].fillna(0, inplace=True)
            rut_df['rut_point_y'].fillna(0, inplace=True)
        
            rut_df.rename(columns={'left rutting height': 'left_rutting', 'right rutting height': 'right_rutting', 'average height': 'avg_rutting'}, inplace=True)
            rut_df.drop(columns=['qgis_shape'], inplace=True)
            rutting_dataframes[filename] = rut_df

            print(f"Updated {filename} into Rutting DataFrame.")

    return iri_dataframes, rutting_dataframes

# Perform the left join on xw_rutting and xw_iri_qgis
def left_join_dataframes(df_rutting, df_iri):
    joined_df = pd.merge(df_rutting, df_iri, how='left', on=['event_start', 'event_end', 'survey_code'], suffixes=('_rutting', '_iri'))
    return joined_df

# Perform jpg file and frame number
def get_jpg_names_and_nums(directory):
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = [f for f in files if f.endswith('.jpg')]
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)

    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace('_', 'RUN0')

    return jpg_files, frame_df

# use

jpg_files, frame_df = get_jpg_names_and_nums(pic)
iri_dataframes, rutting_dataframes = process_csv_files(path)

joined_dataframes = {}
for rutting_file in rutting_dataframes:
    for iri_file in iri_dataframes:
        if 'xw_rutting' in rutting_file and 'xw_iri_qgis' in iri_file:
            joined_df = left_join_dataframes(rutting_dataframes[rutting_file], iri_dataframes[iri_file])

            # Group by survey_code and calculate min and max chainage
            grouped_df = joined_df.groupby('survey_code').agg(
                max_chainage=('chainage', 'max'),
                min_chainage=('chainage', 'min')
            ).reset_index()

            joined_df = pd.merge(joined_df, grouped_df, on='survey_code', how='left')
            joined_df = pd.merge(joined_df, frame_df, on='survey_code', how='left')
            joined_df['frame_num'] = joined_df.groupby(['survey_code', 'pic_count']).cumcount() + 1
            joined_df['chainage_pic'] = round(joined_df['max_chainage'] / joined_df['pic_count'] * joined_df['frame_num'],0)
            joined_dataframes[f"{rutting_file}_{iri_file}"] = joined_df
            
final_df = pd.concat(joined_dataframes.values(), ignore_index=True)
final_df = final_df[final_df['iri'].notnull()]
final_df.to_csv('final_df.csv', index=False)

def find_csv_files(start_dir, prefix='log_'):
    csv_files = []
    for dirpath, dirnames, filenames in os.walk(start_dir):
        for filename in fnmatch.filter(filenames, f'{prefix}*.xlsx'):
            csv_files.append(os.path.join(dirpath, filename))
    return csv_files

log_csv_files = find_csv_files(log)
if log_csv_files:
    log_df = pd.read_excel(log_csv_files[0])

    # Rename columns and clean up column names
    log_df.rename(columns={'ผิว': 'event_name', 'link_id ระบบ': 'section_id'}, inplace=True)
    log_df.columns = log_df.columns.str.strip()

    # Perform the initial merge and filter rows where frame_num is between numb_start and numb_end
    merged_df = pd.merge(final_df, log_df, how='left', on=['survey_code'], suffixes=('_final_df', '_log_df'))
    merged_df = merged_df[(merged_df['frame_num'] >= merged_df['numb_start']) & 
                          (merged_df['frame_num'] <= merged_df['numb_end'])]

    def process_val(df):
        df['chainage'] = df['chainage_pic']
        df['lon'] = df['rut_point_y']
        df['lat'] = df['rut_point_x']
        df['iri_right'] = df['iri right (m/km)']
        df['iri_left'] = df['iri left (m/km)']
        df['iri'] = df['iri']
        df['iri_lane'] = df['iri_lane']
        df['rutt_right'] = df['right_rutting']
        df['rutt_left'] = df['left_rutting']
        df['rutting'] = df['avg_rutting']
        df['texture'] = 0
        df['etd_texture'] = 0
        df['event_name'] = df['event_name'].str.lower()
        df['frame_number'] = df['frame_num']
        df['file_name'] = df['survey_code'].str.replace('RUN0', '_')
        df['run_code'] = df['file_name'].str.split('_').str[-1]

        return df

    processed_val = process_val(merged_df)

    selected_columns_val = [
        'chainage', 'lon', 'lat', 'iri_right', 'iri_left', 'iri', 'iri_lane', 'rutt_right', 'rutt_left', 
        'rutting', 'texture', 'etd_texture', 'event_name', 'frame_number', 'file_name', 'run_code'
    ]

    selected_columns_val = [col for col in selected_columns_val if col in processed_val.columns]
    processed_val[selected_columns_val].to_csv('access_valuelaser.csv', index=False)
    
    # edit 100% correct
    def process_dis(df):
        df['chainage_pic'] = df['chainage_pic']
        df['frame_number'] = df['frame_num']
        df['event_name'] = df['event_name'].str.lower()
        df['name_key'] = df['survey_code'].str.replace('RUN0', '_')
        df['run_code'] = df['file_name'].str.split('_').str[-1]

        return df

    processed_dis = process_dis(merged_df)

    selected_columns_dis = [
        'chainage_pic', 'frame_number', 'event_name', 'name_key', 'run_code'
    ]

    selected_columns_dis = [col for col in selected_columns_dis if col in processed_dis.columns]
    processed_dis[selected_columns_dis].to_csv('access_distress_pic.csv', index=False)
    
    def process_key(df):
        df['event_str'] = round(df['numb_start'] * (df['max_chainage'] / df['pic_count']))
        df['event_end'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count']))
        df['event_num'] = df['event_name'].str[0].str.lower()
        df['event_type'] = 'pave type'
        df['event_name'] = df['event_name'].str.lower()
        df['link_id'] = df['linkid']
        df['lane_no'] = df['linkid'].apply(lambda x: x[11:13])
        df['survey_date'] = df['date']
        df['lat_str'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('first')
        df['lat_end'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('last')
        df['lon_str'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('first')
        df['lon_end'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('last')
        df['name_key'] = df['survey_code'].str.replace('RUN0', '_')
        df['run_code'] = df['name_key'].str.split('_').str[-1]
        
        return df

    processed_key = merged_df.groupby('survey_code', group_keys=False).apply(process_key).reset_index(drop=True)
    processed_key = processed_key.groupby(['linkid', 'survey_date']).first().reset_index()

    selected_columns_key = [
        'event_str', 'event_end', 'event_num', 'event_type', 'event_name', 'link_id', 'section_id', 
        'km_start', 'km_end', 'length', 'lane_no', 'survey_date', 'lat_str', 'lat_end', 'lon_str', 
        'lon_end', 'name_key', 'run_code'
    ]

    selected_columns_key = [col for col in selected_columns_key if col in processed_key.columns]
    processed_key[selected_columns_key].sort_values(by=['run_code', 'event_str', 'event_end'], ascending=[True, True, False]).to_csv('access_key.csv', index=False)

Updated xw_iri_qgis_20240726RUN01.csv into IRI DataFrame.
Updated xw_rutting_20240726RUN01.csv into Rutting DataFrame.
Updated xw_iri_qgis_20240726RUN02.csv into IRI DataFrame.
Updated xw_rutting_20240726RUN02.csv into Rutting DataFrame.
Updated xw_iri_qgis_20240726RUN03.csv into IRI DataFrame.
Updated xw_rutting_20240726RUN03.csv into Rutting DataFrame.


In [2]:
final_df

,#Date,left_rutting,right_rutting,avg_rutting,event_start,event_end,survey_code,rut_point_x,rut_point_y,Date,iri left (m/km),iri Std left (m/km),iri right (m/km),iri Std right (m/km),worst iri (m/km),iri difference (m/km),iri,iri_lane,chainage,max_chainage,min_chainage,pic_count,frame_num,chainage_pic
0,26/07/2024,3.46,5.29,4.38,0,5,20240726RUN01,13.870717,100.451832,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,3.754977,3.542092,0.0,1635.0,0.0,246,1,7.0
1,26/07/2024,1.49,4.30,2.90,5,10,20240726RUN01,13.870672,100.451835,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,3.754977,4.105318,5.0,1635.0,0.0,246,2,13.0
2,26/07/2024,2.35,0.42,1.38,10,15,20240726RUN01,13.870627,100.451839,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,3.754977,3.383927,10.0,1635.0,0.0,246,3,20.0
3,26/07/2024,1.96,0.55,1.25,15,20,20240726RUN01,13.870582,100.451842,26/07/2024,3.824703,2.775389,3.685251,3.140042,3.824703,0.139453,3.754977,4.051758,15.0,1635.0,0.0,246,4,27.0
4,26/07/2024,1.69,0.71,1.20,20,25,20240726RUN01,13.870537,100.451846,26/07/2024,4.101019,3.490062,4.845301,3.479725,4.845301,0.744282,4.473160,4.586364,20.0,1635.0,0.0,246,5,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8322,26/07/2024,0.41,6.50,3.46,5995,6000,20240726RUN03,13.849415,100.443981,26/07/2024,8.690157,8.111755,10.062783,8.875238,10.062783,1.372626,9.376470,9.492709,5995.0,6015.0,0.0,857,1200,8422.0
8323,26/07/2024,0.34,6.09,3.22,6000,6005,20240726RUN03,13.849436,100.443940,26/07/2024,15.840459,12.255976,16.280330,12.531867,16.280330,0.439871,16.060394,16.365209,6000.0,6015.0,0.0,857,1201,8429.0
8324,26/07/2024,5.66,2.46,4.06,6005,6010,20240726RUN03,13.849459,100.443900,26/07/2024,15.840459,12.255976,16.280330,12.531867,16.280330,0.439871,16.060394,17.253204,6005.0,6015.0,0.0,857,1202,8436.0
8325,26/07/2024,6.84,0.27,3.56,6010,6015,20240726RUN03,13.849488,100.443864,26/07/2024,15.840459,12.255976,16.280330,12.531867,16.280330,0.439871,16.060394,14.881872,6010.0,6015.0,0.0,857,1203,8443.0


Find pic and make survey_code and pic_count

In [148]:
# Perform jpg file and frame number
def get_jpg_names_and_nums(directory):
    jpg_dict = {}
    
    for root, dirs, files in os.walk(directory):
        jpg_files = {[f for f in files if f.endswith('.jpg')]}
        if jpg_files:
            folder_name = os.path.basename(os.path.dirname(root))
            jpg_dict[folder_name] = len(jpg_files)

    frame_df = pd.DataFrame(list(jpg_dict.items()), columns=['survey_code', 'pic_count'])
    frame_df['survey_code'] = frame_df['survey_code'].str.replace('_', 'RUN0')

    return jpg_files, frame_df

print(jpg_files)
print(frame_df)

['20240726_3_PAVE-0-00001.jpg', '20240726_3_PAVE-0-00002.jpg', '20240726_3_PAVE-0-00003.jpg', '20240726_3_PAVE-0-00004.jpg', '20240726_3_PAVE-0-00005.jpg', '20240726_3_PAVE-0-00006.jpg', '20240726_3_PAVE-0-00007.jpg', '20240726_3_PAVE-0-00008.jpg', '20240726_3_PAVE-0-00009.jpg', '20240726_3_PAVE-0-00010.jpg', '20240726_3_PAVE-0-00011.jpg', '20240726_3_PAVE-0-00012.jpg', '20240726_3_PAVE-0-00013.jpg', '20240726_3_PAVE-0-00014.jpg', '20240726_3_PAVE-0-00015.jpg', '20240726_3_PAVE-0-00016.jpg', '20240726_3_PAVE-0-00017.jpg', '20240726_3_PAVE-0-00018.jpg', '20240726_3_PAVE-0-00019.jpg', '20240726_3_PAVE-0-00020.jpg', '20240726_3_PAVE-0-00021.jpg', '20240726_3_PAVE-0-00022.jpg', '20240726_3_PAVE-0-00023.jpg', '20240726_3_PAVE-0-00024.jpg', '20240726_3_PAVE-0-00025.jpg', '20240726_3_PAVE-0-00026.jpg', '20240726_3_PAVE-0-00027.jpg', '20240726_3_PAVE-0-00028.jpg', '20240726_3_PAVE-0-00029.jpg', '20240726_3_PAVE-0-00030.jpg', '20240726_3_PAVE-0-00031.jpg', '20240726_3_PAVE-0-00032.jpg', '202407

join final_df and log_xenomatix Toget access_key, access_valuelaser, access_distress_pic

In [154]:
# Load Log data
log = r'D:\xenomatixs\output\survey_data_20240726\Output'

def find_csv_files(start_dir, prefix='log_'):
    csv_files = []
    for dirpath, dirnames, filenames in os.walk(start_dir):
        for filename in fnmatch.filter(filenames, f'{prefix}*.xlsx'):
            csv_files.append(os.path.join(dirpath, filename))
    return csv_files

# Find all .csv files that start with 'log_'
log_csv_files = find_csv_files(path)

# Print the list of found files
for file in log_csv_files:
    print(file)

D:\xenomatixs\output\survey_data_20240726\Output\log_xenomatix_20240726.xlsx


In [7]:
import os
import fnmatch
import pandas as pd

log = r'D:\xenomatixs\output\survey_data_20240726\Output'

def find_csv_files(start_dir, prefix='log_'):
    csv_files = []
    for dirpath, dirnames, filenames in os.walk(start_dir):
        for filename in fnmatch.filter(filenames, f'{prefix}*.xlsx'):
            csv_files.append(os.path.join(dirpath, filename))
    return csv_files

log_csv_files = find_csv_files(log)
if log_csv_files:
    log_df = pd.read_excel(log_csv_files[0])

    # Rename columns and clean up column names
    log_df.rename(columns={'ผิว': 'event_name', 'link_id ระบบ': 'section_id'}, inplace=True)
    log_df.columns = log_df.columns.str.strip()

    # Perform the initial merge and filter rows where frame_num is between numb_start and numb_end
    merged_df = pd.merge(final_df, log_df, how='left', on=['survey_code'], suffixes=('_final_df', '_log_df'))
    merged_df = merged_df[(merged_df['frame_num'] >= merged_df['numb_start']) & 
                          (merged_df['frame_num'] <= merged_df['numb_end'])]

    def process_val(df):
        df['chainage'] = df['chainage_pic']
        df['lon'] = df['rut_point_y']
        df['lat'] = df['rut_point_x']
        df['iri_right'] = df['iri right (m/km)']
        df['iri_left'] = df['iri left (m/km)']
        df['iri'] = df['iri']
        df['iri_lane'] = df['iri_lane']
        df['rutt_right'] = df['right_rutting']
        df['rutt_left'] = df['left_rutting']
        df['rutting'] = df['avg_rutting']
        df['texture'] = 0
        df['etd_texture'] = 0
        df['event_name'] = df['event_name'].str.lower()
        df['frame_number'] = df['frame_num']
        df['file_name'] = df['survey_code'].str.replace('RUN0', '_')
        df['run_code'] = df['file_name'].str.split('_').str[-1]

        return df

    processed_val = process_val(merged_df)

    selected_columns_val = [
        'chainage', 'lon', 'lat', 'iri_right', 'iri_left', 'iri', 'iri_lane', 'rutt_right', 'rutt_left', 
        'rutting', 'texture', 'etd_texture', 'event_name', 'frame_number', 'file_name', 'run_code'
    ]

    selected_columns_val = [col for col in selected_columns_val if col in processed_val.columns]
    processed_val[selected_columns_val].to_csv('access_valuelaser.csv', index=False)
    
    def process_dis(df):
        df['chainage_pic'] = df['chainage_pic']
        df['frame_number'] = df['frame_num']
        df['event_name'] = df['event_name'].str.lower()
        df['name_key'] = df['survey_code'].str.replace('RUN0', '_')
        df['run_code'] = df['file_name'].str.split('_').str[-1]

        return df

    processed_dis = process_dis(merged_df)

    selected_columns_dis = [
        'chainage_pic', 'frame_number', 'event_name', 'name_key', 'run_code'
    ]

    selected_columns_dis = [col for col in selected_columns_dis if col in processed_dis.columns]
    processed_dis[selected_columns_dis].to_csv('access_distress_pic.csv', index=False)
    

    def process_key(df):
        df['event_str'] = round(df['numb_start'] * (df['max_chainage'] / df['pic_count']))
        df['event_end'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count']))
        df['event_num'] = df['event_name'].str[0].str.lower()
        df['event_type'] = 'pave type'
        df['event_name'] = df['event_name'].str.lower()
        df['link_id'] = df['linkid']
        df['lane_no'] = df['linkid'].apply(lambda x: x[11:13])
        df['survey_date'] = df['date']
        df['lat_str'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('first')
        df['lat_end'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('last')
        df['lon_str'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('first')
        df['lon_end'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('last')
        df['name_key'] = df['survey_code'].str.replace('RUN0', '_')
        df['run_code'] = df['name_key'].str.split('_').str[-1]
        
        return df

    processed_key = merged_df.groupby('survey_code', group_keys=False).apply(process_key).reset_index(drop=True)
    processed_key = processed_key.groupby(['linkid', 'survey_date']).first().reset_index()

    selected_columns_key = [
        'event_str', 'event_end', 'event_num', 'event_type', 'event_name', 'link_id', 'section_id', 
        'km_start', 'km_end', 'length', 'lane_no', 'survey_date', 'lat_str', 'lat_end', 'lon_str', 
        'lon_end', 'name_key', 'run_code'
    ]

    selected_columns_key = [col for col in selected_columns_key if col in processed_key.columns]
    processed_key[selected_columns_key].sort_values(by=['run_code', 'event_str', 'event_end'], ascending=[True, True, False]).to_csv('access_key.csv', index=False)

In [8]:
processed_val[selected_columns_val].head(10)

,chainage,lon,lat,iri_right,iri_left,iri,iri_lane,rutt_right,rutt_left,rutting,texture,etd_texture,event_name,frame_number,file_name,run_code
36,246.0,100.451957,13.869095,5.258881,5.559707,5.409294,5.090888,1.67,2.62,2.14,0,0,ac,37,20240726_1,1
37,253.0,100.451960,13.869050,5.258881,5.559707,5.409294,5.684624,1.22,1.84,1.53,0,0,ac,38,20240726_1,1
38,259.0,100.451962,13.869004,5.258881,5.559707,5.409294,5.523508,8.20,0.00,4.10,0,0,ac,39,20240726_1,1
39,266.0,100.451965,13.868959,5.258881,5.559707,5.409294,5.624472,4.52,3.04,3.78,0,0,ac,40,20240726_1,1
40,272.0,100.451967,13.868914,10.746929,12.978930,11.862930,11.515827,0.98,4.35,2.66,0,0,ac,41,20240726_1,1
41,279.0,100.451970,13.868869,10.746929,12.978930,11.862930,12.647515,1.46,2.16,1.81,0,0,ac,42,20240726_1,1
42,286.0,100.451974,13.868824,10.746929,12.978930,11.862930,11.974729,0.65,3.14,1.90,0,0,ac,43,20240726_1,1
43,292.0,100.451977,13.868779,10.746929,12.978930,11.862930,11.264476,1.14,3.87,2.50,0,0,ac,44,20240726_1,1
44,299.0,100.451979,13.868734,3.436835,3.455609,3.446222,3.509292,0.00,2.87,1.44,0,0,ac,45,20240726_1,1
45,306.0,100.451982,13.868689,3.436835,3.455609,3.446222,3.485207,0.00,5.96,2.98,0,0,ac,46,20240726_1,1


Convert .csv to .mdb 

In [ ]:
import os
import fnmatch
import datetime
import pandas as pd

log = r'D:\xenomatixs\output\survey_data_20240726\Output'

def find_csv_files(start_dir, prefix='log_'):
    csv_files = []
    found_filenames = []
    for dirpath, dirnames, filenames in os.walk(start_dir):
        for filename in fnmatch.filter(filenames, f'{prefix}*.xlsx'):
            csv_files.append(os.path.join(dirpath, filename))
            found_filenames.append(filename)  # Store the filenames
    return csv_files, found_filenames

folder_names = [name for name in os.listdir(log) if os.path.isdir(os.path.join(log, name))]
log_csv_files, filenames = find_csv_files(log)

if log_csv_files:
    log_df = pd.read_excel(log_csv_files[0])

    # Rename columns and clean up column names
    log_df.rename(columns={'ผิว': 'event_name', 'link_id ระบบ': 'section_id'}, inplace=True)
    log_df.columns = log_df.columns.str.strip()

    for folder_name in folder_names:
        print(f"Processing folder: {folder_name}")

        current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Perform the initial merge and filter rows where frame_num is between numb_start and numb_end
        merged_df = pd.merge(final_df, log_df, how='left', on=['survey_code'], suffixes=('_final_df', '_log_df'))
        merged_df = merged_df[(merged_df['frame_num'] >= merged_df['numb_start']) & 
                              (merged_df['frame_num'] <= merged_df['numb_end'])]
        
        filtered_df = merged_df[merged_df['survey_code'] == folder_name]
        run_code = folder_name.replace('RUN0', '_')
        
        def mdb_video_process(df):
            df['CHAINAGE'] = df['chainage_pic']
            df['LRP_OFFSET'] = df['chainage_pic']
            df['LRP_NUMBER'] = 0
            df['FRAME'] = df['frame_num']
            df['GPS_TIME'] = 0
            df['X'] = df['rut_point_y']
            df['Y'] = df['rut_point_x']
            df['Z'] = 0
            df['HEADING'] = 0
            df['PITCH'] = 0
            df['ROLL'] = 0

            return df

        video_process = mdb_video_process(filtered_df)
        
        selected_mdb_video_process = [
            'CHAINAGE', 'LRP_OFFSET', 'LRP_NUMBER', 'FRAME', 'GPS_TIME', 
            'X', 'Y', 'Z', 'HEADING', 'PITCH', 'ROLL'
        ]

        selected_mdb_video_process = [col for col in selected_mdb_video_process if col in video_process.columns]
        mdb_video_process_filename = f'D:/xeno/mdb/Video_Processed_{run_code}_2.csv'
        video_process[selected_mdb_video_process].to_csv(mdb_video_process_filename, index=False)
        
        mdb_video_header = pd.DataFrame({
            'CAMERA': [1, 2],
            'NAME': ['ROW-0', 'PAVE-0'],
            'DEVICE': ['XENO', 'XENO'],
            'SERIAL': ['6394983', '6394984'],
            'INTERVAL': [5, 2],
            'WIDTH': [0, 0],
            'HEIGHT': [0, 0],
            'FRAME_RATE': [0, 0],
            'FORMAT': ['422 YUV 8', 'Mono 8'],
            'X_SCALE': [0, 0.5],
            'Y_SCALE': [0, 0.5],
            'DATA_FORMAT': [-1, -1],
            'PROCESSING_METHOD': [-1, -1],
            'ENABLE_MOBILE_MAPPING': [True, False],
            'DISP_PITCH': [0, 0],
            'DISP_ROLL': [0, 0],
            'DISP_YAW': [0, 0],
            'DISP_X': [0, 0],
            'DISP_Y': [0, 0],
            'DISP_Z': [0, 0],
            'HFOV': [0, 0],
            'VFOV': [0, 0]
        })

        mdb_video_header_filename = f'D:/xeno/mdb/Video_Header_{run_code}.csv'
        mdb_video_header.to_csv(mdb_video_header_filename, index=False)
        
        def mdb_survey_header(df):
            ['SURVEY_ID']: [run_code],
            ['SURVEY_FILE']: [run_code],
            ['SURVEY_DESC']: [None],
            ['SURVEY_DATE']: [0],
            ['VEHICLE']: ['ISS'],
            ['OPERATOR']: ['ISS'],
            'USER_1_NAME': [None],
            'USER_1': [None],
            'USER_2_NAME': [None],
            'USER_2': [None],
            'USER_3_NAME': [None],
            'USER_3': [None],
            'LRP_FILE': [f'LRP_{run_code}'],
            'LRP_RESET': ['N'],
            'LRP_START': [0],
            'CHAIN_INIT': [0],
            'CHAIN_START': [0],
            'CHAIN_END': [filtered_df['chainage_end']],
            'SECT_LEN': [0],
            'DIR': ['I'],
            'LANE': [1],
            'DEVICES': ['GPS-Geo-DR,LP_V3-LWP,LP_V3-RWP,TPL,Video'],
            'OTHERSIDE': [True],
            'VERSION': ['2.7.3.4/2.7.3.4'],
            'MEMO': [None],
            'LENGTH': [filtered_df['chainage_end']]

            return df

        survey_header = mdb_video_process(filtered_df)
        
        selected_mdb_survey_header = [
            'SURVEY_ID', 'SURVEY_FILE', 'SURVEY_DESC', 'SURVEY_DATE', 'VEHICLE', 'OPERATOR', 'USER_1_NAME', 'USER_1', 
            'USER_2_NAME', 'USER_2', 'USER_3_NAME', 'USER_3', 'LRP_FILE', 'LRP_RESET', 'LRP_START', 'CHAIN_INIT', 
            'CHAIN_START','CHAIN_END', 'SECT_LEN', 'DIR', 'LANE', 'DEVICES', 'OTHERSIDE', 'VERSION', 'MEMO', 'LENGTH'
        ]

        selected_mdb_survey_header = [col for col in selected_mdb_survey_header if col in survey_header.columns]
        mdb_survey_header_filename = f'D:/xeno/mdb/Video_Processed_{run_code}_2.csv'
        survey_header[selected_mdb_survey_header].to_csv(mdb_survey_header_filename, index=False)
        
        def mdb_KeyCode_Raw(df):
            df['CHAINAGE_START'] = round(df['numb_start'] * (df['max_chainage'] / df['pic_count']))
            df['CHAINAGE_END'] = round(df['numb_end'] * (df['max_chainage'] / df['pic_count']))
            df['EVENT'] = df['event_name'].str[0].str.lower()
            df['SWITCH_GROUP'] = 'pave_type.'
            df['EVENT_DESC'] = df['event_name'].str.lower()
            df['LATITUDE_START'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('first')
            df['LATITUDE_END'] = df.groupby(['survey_code', 'linkid'])['rut_point_x'].transform('last')
            df['LONGITUDE_START'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('first')
            df['LONGITUDE_END'] = df.groupby(['survey_code', 'linkid'])['rut_point_y'].transform('last')
            df['link_id'] = df['linkid']
            df['section_id'] = df['section_id']
            df['km_start'] = df['km_start']
            df['km_end'] = df['km_end']
            df['length'] = df['length']
            df['lane_no'] = df['linkid'].apply(lambda x: x[11:13])
            df['survey_date'] = df['date']
            
            return df

        KeyCode_Raw = merged_df.groupby('survey_code', group_keys=False).apply(mdb_KeyCode_Raw).reset_index(drop=True)
        KeyCode_Raw = KeyCode_Raw.groupby(['linkid', 'survey_date']).first().reset_index()
        KeyCode_Raw = KeyCode_Raw[KeyCode_Raw['survey_code'] == folder_name]

        selected_mdb_KeyCode_Raw = [
            'CHAINAGE_START', 'CHAINAGE_END', 'EVENT', 'SWITCH_GROUP', 'EVENT_DESC', 'LATITUDE_START', 'LATITUDE_END', 
            'LONGITUDE_START', 'LONGITUDE_END', 'link_id', 'section_id', 'km_start', 'km_end', 'length', 'lane_no', 
            'survey_date'
        ]

        selected_mdb_KeyCode_Raw = [col for col in selected_mdb_KeyCode_Raw if col in KeyCode_Raw.columns]
        KeyCode_Raw[selected_mdb_KeyCode_Raw].sort_values(by=['lane_no', 'CHAINAGE_START', 'CHAINAGE_END'], ascending=[True, True, False]).to_csv(f'D:/xeno/mdb/KeyCode_Raw_{run_code}.csv', index=False)        

In [13]:
filtered_df.head(10)

,#Date,left_rutting,right_rutting,avg_rutting,event_start,event_end,survey_code,rut_point_x,rut_point_y,Date,iri left (m/km),iri Std left (m/km),iri right (m/km),iri Std right (m/km),worst iri (m/km),iri difference (m/km),iri,iri_lane,chainage,max_chainage,min_chainage,pic_count,frame_num,chainage_pic,linkid,ramp_id,section_id,numb_start,numb_end,km_start,km_end,length,length_KM,lane,event_name,date,route,remark,CHAINAGE,LRP_OFFSET,LRP_NUMBER,FRAME,GPS_TIME,X,Y,Z,HEADING,PITCH,ROLL
2884,26/07/2024,2.09,0.00,1.04,230,235,20240726RUN03,13.856109,100.490599,26/07/2024,3.689781,2.486528,2.849825,2.027284,3.689781,0.839956,3.269803,3.042229,230.0,6015.0,0.0,857,47,330.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,330.0,330.0,0,47,0,100.490599,13.856109,0,0,0,0
2886,26/07/2024,3.22,0.00,1.61,235,240,20240726RUN03,13.856106,100.490553,26/07/2024,3.689781,2.486528,2.849825,2.027284,3.689781,0.839956,3.269803,3.542747,235.0,6015.0,0.0,857,48,337.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,337.0,337.0,0,48,0,100.490553,13.856106,0,0,0,0
2888,26/07/2024,3.91,0.00,1.96,240,245,20240726RUN03,13.856101,100.490507,26/07/2024,3.301049,3.351751,3.033456,2.591172,3.301049,0.267593,3.167252,3.093392,240.0,6015.0,0.0,857,49,344.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,344.0,344.0,0,49,0,100.490507,13.856101,0,0,0,0
2890,26/07/2024,2.50,0.92,1.71,245,250,20240726RUN03,13.856095,100.490461,26/07/2024,3.301049,3.351751,3.033456,2.591172,3.301049,0.267593,3.167252,3.069261,245.0,6015.0,0.0,857,50,351.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,351.0,351.0,0,50,0,100.490461,13.856095,0,0,0,0
2892,26/07/2024,2.03,0.00,1.01,250,255,20240726RUN03,13.856088,100.490415,26/07/2024,3.301049,3.351751,3.033456,2.591172,3.301049,0.267593,3.167252,3.333163,250.0,6015.0,0.0,857,51,358.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,358.0,358.0,0,51,0,100.490415,13.856088,0,0,0,0
2894,26/07/2024,3.16,0.00,1.58,255,260,20240726RUN03,13.856081,100.490370,26/07/2024,3.301049,3.351751,3.033456,2.591172,3.301049,0.267593,3.167252,3.455302,255.0,6015.0,0.0,857,52,365.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,365.0,365.0,0,52,0,100.490370,13.856081,0,0,0,0
2896,26/07/2024,1.89,0.00,0.94,260,265,20240726RUN03,13.856071,100.490324,26/07/2024,2.730654,3.016062,2.768365,1.909563,2.768365,0.037710,2.749510,2.980129,260.0,6015.0,0.0,857,53,372.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,372.0,372.0,0,53,0,100.490324,13.856071,0,0,0,0
2898,26/07/2024,1.51,0.00,0.76,265,270,20240726RUN03,13.856061,100.490279,26/07/2024,2.730654,3.016062,2.768365,1.909563,2.768365,0.037710,2.749510,2.750638,265.0,6015.0,0.0,857,54,379.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,379.0,379.0,0,54,0,100.490279,13.856061,0,0,0,0
2900,26/07/2024,2.71,0.00,1.36,270,275,20240726RUN03,13.856050,100.490234,26/07/2024,2.730654,3.016062,2.768365,1.909563,2.768365,0.037710,2.749510,2.663732,270.0,6015.0,0.0,857,55,386.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,386.0,386.0,0,55,0,100.490234,13.856050,0,0,0,0
2902,26/07/2024,1.33,0.00,0.66,275,280,20240726RUN03,13.856038,100.490190,26/07/2024,2.730654,3.016062,2.768365,1.909563,2.768365,0.037710,2.749510,2.819204,275.0,6015.0,0.0,857,56,393.0,01900990008L1AC05,191,101641,47,252,0+000,2+520,1435,1.435,L2,AC,20240726,ส.008,NaN,393.0,393.0,0,56,0,100.490190,13.856038,0,0,0,0


In [12]:
import os
import csv
import pyodbc
import win32com.client

def create_access_db(db_path):
    if os.path.isfile(db_path):
        print(f"File already exists: {db_path}")
    else:
        access_app = win32com.client.Dispatch("Access.Application")
        access_app.NewCurrentDatabase(db_path)
        print(f"Created new Access database at: {db_path}")
        access_app.Quit()

def insert_csv_to_access(csv_path, table_name, access_db_path):
    conn_str = r"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={};".format(access_db_path)
    con = None
    try:
        con = pyodbc.connect(conn_str)
        strSQL = (f"SELECT * INTO [{table_name}] "
                  f"FROM [text;HDR=Yes;FMT=Delimited(,);Database={os.path.dirname(csv_path)}].{os.path.basename(csv_path)};")
        cur = con.cursor()
        cur.execute(strSQL)
        con.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if con:
            con.close()



log = r'D:\xenomatixs\output\survey_data_20240726\Output'
patmdb = r'D:\xeno\mdb'

folder_names = [name for name in os.listdir(log) if os.path.isdir(os.path.join(log, name))]

for folder_name in folder_names:
    run_code = folder_name.replace('RUN0', '_')
    mdb_path = os.path.join(patmdb, f'{run_code}_edit.mdb')
    create_access_db(mdb_path)

    # Define CSV files and corresponding table names
    csv_files = {
        f'KeyCode_Raw_{run_code}.csv': f'KeyCode_Raw_{run_code}', 
        # f'Survey_Header_{run_code}.csv': f'Survey_Header_{run_code}',
        f'Survey_Header_{run_code}.csv': f'Survey_Header',
        f'Video_Header_{run_code}.csv': f'Video_Header_{run_code}',
        f'Video_Processed_{run_code}_2.csv': f'Video_Processed_{run_code}_2'
    }

    for csv_name, table_name in csv_files.items():
        csv_path = os.path.join(patmdb, csv_name)
        
        # Insert cleaned CSV into Access database
        insert_csv_to_access(csv_path, table_name, mdb_path)
        os.remove(csv_path)
    print(f"All CSV files have been processed and imported into the Access database.")

Created new Access database at: D:\xeno\mdb\20240726_1_edit.mdb
All CSV files have been processed and imported into the Access database.
Created new Access database at: D:\xeno\mdb\20240726_2_edit.mdb
All CSV files have been processed and imported into the Access database.
Created new Access database at: D:\xeno\mdb\20240726_3_edit.mdb
All CSV files have been processed and imported into the Access database.
